<a href="https://colab.research.google.com/github/gandhimonil9823/CS_GO_Analysis/blob/main/XGBoost_and_AdaBoost_classifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import make_scorer, confusion_matrix, classification_report, precision_score, r2_score, f1_score, recall_score

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from xgboost import XGBClassifier

import time

### Mount data from Google Drive

In [3]:
#mount data folder from google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Change dir and check contents

In [4]:
cd '/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/aggregated_data/modelinput'

/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/aggregated_data/modelinput


In [5]:
ls

winner_matches_agg.csv  winner_side_rounds_agg.csv  winner_team_rounds_agg.csv


### load data


In [6]:
%%time
pred_team_matches = pd.read_csv('/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/aggregated_data/modelinput/winner_matches_agg.csv')
pred_team_rounds = pd.read_csv('/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/aggregated_data/modelinput/winner_side_rounds_agg.csv')
pred_side_rounds = pd.read_csv('/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/aggregated_data/modelinput/winner_team_rounds_agg.csv')

CPU times: user 1.52 s, sys: 207 ms, total: 1.73 s
Wall time: 2.87 s


In [7]:
pred_team_matches

,num_rounds,avg_round_duration,avg_equip_val,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke,winner_class
0,25,84.747674,20009.375000,1,0,0,0,0,0,0,0,-1
1,39,97.655995,21298.717949,0,1,0,0,0,0,0,0,-1
2,29,94.112689,18600.000000,0,0,1,0,0,0,0,0,-1
3,30,94.452687,19579.166667,0,0,0,1,0,0,0,0,1
4,29,102.916448,18168.103448,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
13456,29,81.956562,18927.586207,0,1,0,0,0,0,0,0,1
13457,24,74.316426,18516.304348,0,0,0,0,0,1,0,0,1
13458,18,87.696972,16775.000000,0,0,1,0,0,0,0,0,1
13459,24,75.069213,18711.458333,0,0,0,0,1,0,0,0,1


In [8]:
pred_team_rounds

,round,start_seconds,end_seconds,round_duration,ct_eq_val,t_eq_val,final_ct_alive,final_t_alive,is_bomb_planted,bomb_site,past_halftime,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke,PISTOL_ROUND,ECO,SEMI_ECO,NORMAL,FORCE_BUY,t_lethal_nades_eff,ct_lethal_nades_eff,t_util_nades_eff,ct_util_nades_eff,winner_side_class
0,1,94.30782,160.9591,66.65128,4300,4250,0,1,1,-1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,-1
1,2,160.95910,279.3998,118.44070,6300,19400,1,2,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,3,2,1,2,-1
2,3,279.39980,341.0084,61.60860,7650,19250,0,4,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,-1
3,4,341.00840,435.4259,94.41750,24900,23400,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,2,10,8,1,-1
4,5,435.42590,484.2398,48.81390,5400,20550,3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,18,1647.20600,1694.6550,47.44900,4500,21950,0,3,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,3,0,-1
340329,19,1694.65500,1753.4380,58.78300,16400,26100,0,3,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,3,2,6,3,-1
340330,20,1753.43800,1803.6580,50.22000,15100,26000,0,3,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,4,4,-1
340331,21,1803.65800,1856.0850,52.42700,17550,25500,0,2,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,3,3,2,3,-1


In [9]:
pred_side_rounds

,round,start_seconds,end_seconds,round_duration,ct_eq_val,t_eq_val,final_ct_alive,final_t_alive,is_bomb_planted,bomb_site,past_halftime,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke,PISTOL_ROUND,ECO,SEMI_ECO,NORMAL,FORCE_BUY,t_lethal_nades_eff,ct_lethal_nades_eff,t_util_nades_eff,ct_util_nades_eff,winner_class
0,1,94.30782,160.9591,66.65128,4300,4250,0,1,1,-1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,-1
1,2,160.95910,279.3998,118.44070,6300,19400,1,2,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,3,2,1,2,-1
2,3,279.39980,341.0084,61.60860,7650,19250,0,4,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,-1
3,4,341.00840,435.4259,94.41750,24900,23400,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,2,10,8,1,-1
4,5,435.42590,484.2398,48.81390,5400,20550,3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,18,1647.20600,1694.6550,47.44900,4500,21950,0,3,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,3,0,1
340329,19,1694.65500,1753.4380,58.78300,16400,26100,0,3,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,3,2,6,3,1
340330,20,1753.43800,1803.6580,50.22000,15100,26000,0,3,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,4,4,1
340331,21,1803.65800,1856.0850,52.42700,17550,25500,0,2,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,3,3,2,3,1


### apply standard scaler

In [13]:
stdscl = StandardScaler()
stdscl

StandardScaler(copy=True, with_mean=True, with_std=True)

In [14]:
pred_team_matches.columns

Index(['num_rounds', 'avg_round_duration', 'avg_equip_val', 'de_overpass',
       'de_cache', 'de_mirage', 'de_inferno', 'de_train', 'de_dust2',
       'de_cbble', 'de_nuke', 'winner_class'],
      dtype='object')

In [17]:
std_pred_team_matches = pd.DataFrame(data=StandardScaler().fit_transform(pred_team_matches[['num_rounds', 'avg_round_duration', 'avg_equip_val']]),columns=['num_rounds', 'avg_round_duration', 'avg_equip_val'])
std_pred_team_matches[['de_overpass','de_cache', 'de_mirage', 'de_inferno', 'de_train', 'de_dust2','de_cbble', 'de_nuke', 'winner_class']] = pred_team_matches[['de_overpass','de_cache', 'de_mirage', 'de_inferno', 'de_train', 'de_dust2','de_cbble', 'de_nuke', 'winner_class']]
std_pred_team_matches

,num_rounds,avg_round_duration,avg_equip_val,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke,winner_class
0,-0.114617,-0.323955,1.319312,1,0,0,0,0,0,0,0,-1
1,3.102073,1.318159,2.518032,0,1,0,0,0,0,0,0,-1
2,0.804437,0.867402,0.008996,0,0,1,0,0,0,0,0,-1
3,1.034201,0.910655,0.919341,0,0,0,1,0,0,0,0,1
4,0.804437,1.987361,-0.392544,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
13456,0.804437,-0.679023,0.313558,0,1,0,0,0,0,0,0,1
13457,-0.344381,-1.650952,-0.068817,0,0,0,0,0,1,0,0,1
13458,-1.722962,0.051236,-1.687732,0,0,1,0,0,0,0,0,1
13459,-0.344381,-1.555188,0.112620,0,0,0,0,1,0,0,0,1


In [18]:
pred_team_rounds.columns

Index(['round', 'start_seconds', 'end_seconds', 'round_duration', 'ct_eq_val',
       't_eq_val', 'final_ct_alive', 'final_t_alive', 'is_bomb_planted',
       'bomb_site', 'past_halftime', 'de_overpass', 'de_cache', 'de_mirage',
       'de_inferno', 'de_train', 'de_dust2', 'de_cbble', 'de_nuke',
       'PISTOL_ROUND', 'ECO', 'SEMI_ECO', 'NORMAL', 'FORCE_BUY',
       't_lethal_nades_eff', 'ct_lethal_nades_eff', 't_util_nades_eff',
       'ct_util_nades_eff', 'winner_side_class'],
      dtype='object')

In [19]:
scaled_cols = ['round', 'start_seconds', 'end_seconds', 'round_duration', 'ct_eq_val','t_eq_val','t_lethal_nades_eff', 'ct_lethal_nades_eff', 't_util_nades_eff', 'ct_util_nades_eff']
unscaled_cols = ['is_bomb_planted','bomb_site','de_overpass', 'de_cache', 'de_mirage', 'de_inferno', 'de_train', 'de_dust2', 'de_cbble', 'de_nuke', 'PISTOL_ROUND', 'ECO', 'SEMI_ECO', 'NORMAL', 'FORCE_BUY','winner_side_class']

std_pred_team_matches = pd.DataFrame(data=StandardScaler().fit_transform(pred_team_rounds[scaled_cols]),columns=scaled_cols)
std_pred_team_matches[unscaled_cols] = pred_team_rounds[unscaled_cols]
std_pred_team_matches

,round,start_seconds,end_seconds,round_duration,ct_eq_val,t_eq_val,t_lethal_nades_eff,ct_lethal_nades_eff,t_util_nades_eff,ct_util_nades_eff,is_bomb_planted,bomb_site,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke,PISTOL_ROUND,ECO,SEMI_ECO,NORMAL,FORCE_BUY,winner_side_class
0,-1.584582,-1.581176,-1.606920,-0.734664,-1.399625,-1.609951,-1.061526,-0.851189,-1.498029,-0.993525,1,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,-1
1,-1.459545,-1.490514,-1.446071,1.091897,-1.209110,0.119316,0.282583,-0.389366,-1.140547,-0.615697,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,-1
2,-1.334508,-1.329405,-1.362403,-0.912514,-1.080513,0.102194,-0.613489,-0.851189,-1.140547,-1.371353,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,-1
3,-1.209471,-1.245602,-1.234178,0.244623,0.562679,0.575888,-0.165453,1.457926,1.361827,-0.993525,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,-1
4,-1.084434,-1.117171,-1.167886,-1.363770,-1.294842,0.250580,-0.613489,-0.851189,-0.783065,-0.993525,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,0.541048,0.531151,0.475929,-1.411909,-1.380574,0.410380,-0.613489,-0.851189,-0.425583,-1.371353,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,-1
340329,0.666085,0.595693,0.555760,-1.012170,-0.247010,0.884074,0.282583,-0.389366,0.646863,-0.237869,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,-1
340330,0.791123,0.675652,0.623962,-1.314179,-0.370845,0.872660,-1.061526,-0.620278,-0.068101,0.139959,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,-1
340331,0.916160,0.743964,0.695160,-1.236340,-0.137464,0.815588,0.282583,-0.158455,-0.783065,-0.237869,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,-1


In [20]:
pred_side_rounds.columns

Index(['round', 'start_seconds', 'end_seconds', 'round_duration', 'ct_eq_val',
       't_eq_val', 'final_ct_alive', 'final_t_alive', 'is_bomb_planted',
       'bomb_site', 'past_halftime', 'de_overpass', 'de_cache', 'de_mirage',
       'de_inferno', 'de_train', 'de_dust2', 'de_cbble', 'de_nuke',
       'PISTOL_ROUND', 'ECO', 'SEMI_ECO', 'NORMAL', 'FORCE_BUY',
       't_lethal_nades_eff', 'ct_lethal_nades_eff', 't_util_nades_eff',
       'ct_util_nades_eff', 'winner_class'],
      dtype='object')

In [21]:
scaled_cols = ['round', 'start_seconds', 'end_seconds', 'round_duration', 'ct_eq_val','t_eq_val','t_lethal_nades_eff', 'ct_lethal_nades_eff', 't_util_nades_eff', 'ct_util_nades_eff']
unscaled_cols = ['is_bomb_planted','bomb_site','de_overpass', 'de_cache', 'de_mirage', 'de_inferno', 'de_train', 'de_dust2', 'de_cbble', 'de_nuke', 'PISTOL_ROUND', 'ECO', 'SEMI_ECO', 'NORMAL', 'FORCE_BUY','winner_class']

std_pred_side_rounds = pd.DataFrame(data=StandardScaler().fit_transform(pred_side_rounds[scaled_cols]),columns=scaled_cols)
std_pred_side_rounds[unscaled_cols] = pred_side_rounds[unscaled_cols]
std_pred_side_rounds

,round,start_seconds,end_seconds,round_duration,ct_eq_val,t_eq_val,t_lethal_nades_eff,ct_lethal_nades_eff,t_util_nades_eff,ct_util_nades_eff,is_bomb_planted,bomb_site,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke,PISTOL_ROUND,ECO,SEMI_ECO,NORMAL,FORCE_BUY,winner_class
0,-1.584582,-1.581176,-1.606920,-0.734664,-1.399625,-1.609951,-1.061526,-0.851189,-1.498029,-0.993525,1,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,-1
1,-1.459545,-1.490514,-1.446071,1.091897,-1.209110,0.119316,0.282583,-0.389366,-1.140547,-0.615697,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,-1
2,-1.334508,-1.329405,-1.362403,-0.912514,-1.080513,0.102194,-0.613489,-0.851189,-1.140547,-1.371353,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,-1
3,-1.209471,-1.245602,-1.234178,0.244623,0.562679,0.575888,-0.165453,1.457926,1.361827,-0.993525,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,-1
4,-1.084434,-1.117171,-1.167886,-1.363770,-1.294842,0.250580,-0.613489,-0.851189,-0.783065,-0.993525,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,0.541048,0.531151,0.475929,-1.411909,-1.380574,0.410380,-0.613489,-0.851189,-0.425583,-1.371353,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1
340329,0.666085,0.595693,0.555760,-1.012170,-0.247010,0.884074,0.282583,-0.389366,0.646863,-0.237869,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1
340330,0.791123,0.675652,0.623962,-1.314179,-0.370845,0.872660,-1.061526,-0.620278,-0.068101,0.139959,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
340331,0.916160,0.743964,0.695160,-1.236340,-0.137464,0.815588,0.282583,-0.158455,-0.783065,-0.237869,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1


### set up hyperparameter search and train test split

In [43]:
def ttSplit(df,test_size,random_state):

  data = df.iloc[:, :-1]
  target = df.iloc[:,-1:]
  x_train, x_test, y_train, y_test = train_test_split(data, target, test_size=test_size, random_state=random_state)

  return x_train, x_test, y_train, y_test

In [44]:
def hyperparameter_exhaustive_search (model, parameters, scoring, x_train, x_test, y_train, y_test):
  for i in range (len(scoring)):

        startTime = time.time()

        clf = GridSearchCV(estimator=model, param_grid=parameters, scoring=scoring[i],n_jobs=-1)
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)

        endTime = time.time()

        clf_report = classification_report(y_test,y_pred)

        print('Optimizing hyperparameters for ', scoring[i])
        print('\nElapsed Time (s):', round((endTime -startTime),3))
        
        print("\nBest parameters:", clf.best_params_)

        print('\nClassification report:\n', clf_report)

        print('-----------------------------------------------------------------------------\n')

  print('Search Complete!')

### XGBoost

In [45]:
def find_best_XGB(x_train, x_test, y_train, y_test):
  model = XGBClassifier()
  xgbparameters = {'learning_rate': [0.1, 0.2, 1], 'max_depth': [1,2,3], 'n_estimators': [10, 50, 100]}

  #the average function is binary by default. This causes error in the GridSearchCV function for Decision Trees. therefore pass a score function with custom average settings
  average_method = 'micro'  # None, ‘binary’ (default), ‘micro’, ‘macro’, ‘samples’, ‘weighted’
  scoring =[
      make_scorer(f1_score,average=average_method)
      ]

  hyperparameter_exhaustive_search(model=model, parameters=xgbparameters, scoring=scoring, x_train=x_train, x_test=x_test, y_train=y_train, y_test=y_test)

In [49]:
x_train, x_test, y_train, y_test = ttSplit(df=std_pred_team_matches[:50000],test_size=0.3,random_state=0) #only the first 50k lines. takes forver otherwise
find_best_XGB(x_train, x_test, y_train, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Optimizing hyperparameters for  make_scorer(f1_score, average=micro)

Elapsed Time (s): 136.098

Best parameters: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 100}

Classification report:
               precision    recall  f1-score   support

          -1       0.78      0.83      0.81      7665
           1       0.81      0.76      0.78      7335

    accuracy                           0.80     15000
   macro avg       0.80      0.80      0.80     15000
weighted avg       0.80      0.80      0.80     15000

-----------------------------------------------------------------------------

Search Complete!
